In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import random

Sampling Reddit Posts 

In [42]:
# Read and sample from train_pair_data.jsonlist
with open('./pair_task/train_pair_data.jsonlist', 'r') as file:
    train_lines = file.readlines()
sampled_train_lines = random.sample(train_lines, 518)

# Read and sample from heldout_pair_data.jsonlist
with open('./pair_task/heldout_pair_data.jsonlist', 'r') as file:
    heldout_lines = file.readlines()
sampled_heldout_lines = random.sample(heldout_lines, 121)

# Combine sampled lines
combined_sampled_lines = sampled_train_lines + sampled_heldout_lines

# Write combined lines to a new file
with open('./pair_task/combined.jsonlist', 'w') as file:
    for line in combined_sampled_lines:
        file.write(line)


Pre-Processing

In [2]:
filepath = './pair_task/combined.jsonlist'

def process_json_file(file_path, feature):
    all_comments_data = []
    direct_comments_data = []

    with open(file_path, 'r') as file:
        count = 0
        for line in file:
            post = json.loads(line.strip())

            # Replace newline characters with space
            op_title = post['op_title'].replace('\n', ' ')
            op_text = post['op_text'].replace('\n', ' ')

            # Process and join comments, replacing newlines with space
            all_pos_comments = " ".join([comment[feature].replace('\n', ' ') for comment in post['positive']['comments']])
            all_neg_comments = " ".join([comment[feature].replace('\n', ' ') for comment in post['negative']['comments']])

            if np.random.rand() > 0.5:
                all_comments_data.append((op_title, op_text, all_pos_comments, all_neg_comments, 0))
            else:
                all_comments_data.append((op_title, op_text, all_neg_comments, all_pos_comments, 1))

            if post['positive']['comments'] and post['positive']['comments'][0]['parent_id'].startswith('t3_'):
                pos_comment = post['positive']['comments'][0][feature].replace('\n', ' ')
                if post['negative']['comments'] and post['negative']['comments'][0]['parent_id'].startswith('t3_'):
                    neg_comment = post['negative']['comments'][0][feature].replace('\n', ' ')
                    if np.random.rand() > 0.5:
                        direct_comments_data.append((op_title, op_text, pos_comment, neg_comment, 0))
                    else:
                        direct_comments_data.append((op_title, op_text, neg_comment, pos_comment, 1))

            if all_pos_comments == pos_comment and all_neg_comments == neg_comment:
                count += 1

    df_all_comments = pd.DataFrame(all_comments_data, columns=['op_title', 'op_text', 'comment_0', 'comment_1', 'correct'])
    df_direct_comments = pd.DataFrame(direct_comments_data, columns=['op_title', 'op_text', 'comment_0', 'comment_1', 'correct'])

    return df_all_comments, df_direct_comments, count

In [6]:
df_all_comments, df_direct_comments, count = process_json_file(filepath, 'body')
df_all_comments_html, df_direct_comments_html, count = process_json_file(filepath, 'body_html')

In [7]:
count

381

In [48]:
df_all_comments.to_csv('./clean_data/all_comments.csv', index=False)
df_direct_comments.to_csv('./clean_data/direct_comments.csv', index=False)
df_all_comments_html.to_csv('./clean_data/all_comments_html.csv', index=False)
df_direct_comments_html.to_csv('./clean_data/direct_comments_html.csv', index=False)